# Setup

In [1]:
from google.colab import drive

drive.mount("/content/drive")

# Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a EECS595 folder and put all the files under HW1 folder, then 'EECS595/HW1'

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #
#           TODO: Change this to your project path            #
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = "CSE 595/FinalProject"

# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = #

Mounted at /content/drive


In [2]:
import sys
import os
GOOGLE_DRIVE_PATH = os.path.join("drive", "My Drive", GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
sys.path.append(GOOGLE_DRIVE_PATH)
# www_path = os.path.join(GOOGLE_DRIVE_PATH, 'www')
# sys.path.append(www_path)
# dataset_path = os.path.join(GOOGLE_DRIVE_PATH, 'www/dataset')
# sys.path.append(dataset_path)
# model_path = os.path.join(GOOGLE_DRIVE_PATH, 'www/model')
# sys.path.append(model_path)

print(os.listdir(GOOGLE_DRIVE_PATH))
print(os.listdir(os.path.join(GOOGLE_DRIVE_PATH, "www")))
# print(os.listdir(www_path))
# print(os.listdir(dataset_path))
# print(os.listdir(model_path))

# print(GOOGLE_DRIVE_PATH)

['data', 'cache', 'www', 'TODO.gdoc', 'saved_models', 'BertFeaturescopy.ipynb', 'baseline.ipynb', 'spacey.ipynb', 'CSE 595 Final Presentation.gslides', 'FinalProject.ipynb']
['utils.py', 'dataset', 'model', '__pycache__']


In [3]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', 'drive/My Drive/CSE 595/FinalProject']


In [4]:
mode = 'bert' # BERT large

In [5]:
task_name = 'ce'

In [6]:
debug = False

In [7]:
config_batch_size = 32
config_lr = 7.5e-6 # Selected learning rate for best BERT-based model in TRIP paper
config_epochs = 8

For evaluating models

In [31]:
eval_model_dir = 'bert-large-uncased_ConvEnt_32_7.5e-06_7_xval'
# eval_model_dir = 'bert-large-uncased_ConvEnt_1_1e-05_0_xval'
# eval_model_dir = 'bert-large-uncased_ConvEnt_32_1e-05_7_xval'

##Colab Setup

In [9]:
%load_ext autoreload
%autoreload 2

##Model Setup

In [10]:
!pip install coreferee
!pip install spacy
!pip install spacy-transformers
!python -m coreferee install en
!python3 -m spacy download en_core_web_trf
!python3 -m spacy download en_core_web_lg
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.6/182.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1
  Attempting uninstall: smart-open
    Found existing installation: smart-open 7.0.5
    Uninstalling smart-open-7.0.5:
      Successfully uninstalled smart-open-7.0.5
  Attempting uninstall: pydantic
    Found exist

In [11]:
!pip install jsonlines
!pip install transformers
!pip install sentencepiece
!pip3 install torch torchvision torchaudio

In [12]:
import json
import torch
import random
import numpy as np
import spacy_transformers
import coreferee
import spacy

Model Components

In [28]:
if task_name in ['trip', 'ce']:
  multiple_choice = False
elif task_name == 'art':
  multiple_choice = True
else:
  raise ValueError("Task name should be set to 'trip', 'ce', or 'art' in the first cell of the notebook!")

if mode == 'bert':
  model_name = 'bert-large-uncased'

Model tokenizer

In [29]:
from transformers import BertTokenizer
# from DeBERTa import deberta
if mode in ['bert']:
  tokenizer_class = BertTokenizer

tokenizer = tokenizer_class.from_pretrained(model_name,
                                                do_lower_case = False,
                                                cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Load model and optimizer

In [30]:
from transformers import BertForSequenceClassification, AlbertForSequenceClassification, AdamW
from transformers import BertModel, AlbertModel, T5Model, T5EncoderModel, GPT2Model
from transformers import BertConfig, AlbertConfig, T5Config, GPT2Config
from torch.optim import Adam

if mode == 'bert':
  model_class = BertForSequenceClassification
  config_class = BertConfig
  emb_class = BertModel

#Baseline BERT

##Load Conversational Entailment Data

In [32]:
import xml.etree.ElementTree as ET
import pickle
cache_train = os.path.join(GOOGLE_DRIVE_PATH, 'data/ConvEnt_train_resplit.json')
cache_dev = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_resplit.json')
cache_test = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_test_resplit.json')
ConvEnt_train = json.load(open(cache_train))
ConvEnt_dev = json.load(open(cache_dev))
ConvEnt_test = json.load(open(cache_test))

# Combine train and dev and do cross-validation
cache_folds = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_folds.pkl') # Folds used for results presented in paper
ConvEnt_train = ConvEnt_train + ConvEnt_dev
train_sources = list(set([ex['dialog_source'] for ex in ConvEnt_train]))
print("Reserved %s dialog sources for training and validation." % len(train_sources))

no_folds = 8
if not os.path.exists(cache_folds):
  folds = []
  for k in range(no_folds):
    folds.append(np.random.choice(train_sources, size=5, replace=False))
    train_sources = [s for s in train_sources if s not in folds[-1]]
  assert len(train_sources) == 0
  print(folds)
  pickle.dump(folds, open(cache_folds, 'wb'))
else:
  folds = pickle.load(open(cache_folds, 'rb'))

Reserved 40 dialog sources for training and validation.


In [33]:
print('train examples:', len(ConvEnt_train))
print('dev examples:', len(ConvEnt_dev))
print('test examples:', len(ConvEnt_test))

train examples: 703
dev examples: 110
test examples: 172


##Featurize Data

In [34]:
from www.dataset.featurize import add_bert_features_ConvEnt, get_tensor_dataset
import pickle
seq_length = 128

ConvEnt_train = add_bert_features_ConvEnt(ConvEnt_train, tokenizer, seq_length, add_segment_ids=True)
ConvEnt_dev = add_bert_features_ConvEnt(ConvEnt_dev, tokenizer, seq_length, add_segment_ids=True)
ConvEnt_test = add_bert_features_ConvEnt(ConvEnt_test, tokenizer, seq_length, add_segment_ids=True)

ConvEnt_train_folds = [[] for _ in range(no_folds)]
ConvEnt_dev_folds = [[] for _ in range(no_folds)]
for k in range(no_folds):
  ConvEnt_train_folds[k] = [ex for ex in ConvEnt_train if ex['dialog_source'] not in folds[k]]
  ConvEnt_dev_folds[k] = [ex for ex in ConvEnt_train if ex['dialog_source'] in folds[k]]

  if debug:
    ConvEnt_train_folds[k] = ConvEnt_train_folds[k][:10]
    ConvEnt_dev_folds[k] = ConvEnt_dev_folds[k][:10]

if debug:
  ConvEnt_train = ConvEnt_train[:10]
  ConvEnt_dev = ConvEnt_dev[:10]
  ConvEnt_test = ConvEnt_test[:10]

ConvEnt_train_tensor = get_tensor_dataset(ConvEnt_train, label_key='label', add_segment_ids=True)
ConvEnt_test_tensor = get_tensor_dataset(ConvEnt_test, label_key='label', add_segment_ids=True)

# Training sets for each validation fold
ConvEnt_train_folds_tensor = [get_tensor_dataset(ConvEnt_train_folds[k], label_key='label', add_segment_ids=True) for k in range(no_folds)]
ConvEnt_dev_folds_tensor = [get_tensor_dataset(ConvEnt_dev_folds[k], label_key='label', add_segment_ids=True) for k in range(no_folds)]


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [35]:
print('train examples:', len(ConvEnt_train))
print('dev examples:', len(ConvEnt_dev))
print('test examples:', len(ConvEnt_test))

train examples: 703
dev examples: 110
test examples: 172


##Train Model

In [36]:
batch_sizes = [config_batch_size]
learning_rates = [config_lr]
epochs = config_epochs
eval_batch_size = 128

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.train import train_epoch
from www.model.eval import evaluate, save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir
from collections import Counter

seed_val = 22 # Save random seed for reproducibility
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

assert len(batch_sizes) == 1
train_fold_sampler = [RandomSampler(f) for f in ConvEnt_train_folds_tensor]
train_fold_dataloader = [DataLoader(f, sampler=train_fold_sampler[i], batch_size=batch_sizes[0]) for i, f in enumerate(ConvEnt_train_folds_tensor)]

dev_fold_sampler = [SequentialSampler(f) for f in ConvEnt_dev_folds_tensor]
dev_fold_dataloader = [DataLoader(f, sampler=dev_fold_sampler[i], batch_size=eval_batch_size) for i, f in enumerate(ConvEnt_dev_folds_tensor)]

all_val_accs = Counter()
print('Beginning grid search for ConvEnt over %s parameter combination(s)!' % (str(len(batch_sizes) * len(learning_rates))))
for bs in batch_sizes:
  for lr in learning_rates:
    print('\nTRAINING MODEL: bs=%s, lr=%s' % (str(bs), str(lr)))

    for k in range(no_folds):
      print('Beginning fold %s/%s...' % (str(k+1), str(no_folds)))

      # Set up model
      if 'mnli' not in mode:
        model = model_class.from_pretrained(model_name,
                                            cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
      else:
        config = config_class.from_pretrained(model_name.replace('-mnli',''),
                                        num_labels=3,
                                        cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
        model = model_class.from_pretrained(model_name,
                                            config=config,
                                            cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
        config.num_labels = 2
        model.num_labels = 2
        model.classifier = cls_head_class(config=config) # Need to bring in a classification head for only 2 labels

      model.cuda()
      device = model.device

      # Set up optimizer
      optimizer = AdamW(model.parameters(), lr=lr)
      total_steps = len(train_fold_dataloader[k]) * epochs
      scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps = total_steps)

      for epoch in range(epochs):
        # Train the model for one epoch
        print('[%s] Beginning epoch...' % str(epoch))

        epoch_loss, _ = train_epoch(model, optimizer, train_fold_dataloader[k], device, seg_mode=True if 'roberta' not in mode else False)

        # Validate on dev set
        results, _, _ = evaluate(model, dev_fold_dataloader[k], device, [(accuracy_score, 'accuracy')], seg_mode=True if 'roberta' not in mode else False)
        print('[%s] Validation results:' % str(epoch))
        print_dict(results)

        # Save accuracy
        acc = results['accuracy']
        if (bs, lr, epoch) in all_val_accs:
          all_val_accs[(bs, lr, epoch)] += acc
        else:
          all_val_accs[(bs, lr, epoch)] = acc

      model.cpu()
      del model
      del optimizer
      del results
      del scheduler
      del total_steps

      print('[%s] Finished epoch.' % str(epoch))

for k in all_val_accs:
  all_val_accs[k] /= no_folds

print('Top performing param combos:')
print(all_val_accs.most_common(5))

save_fname = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models/%s_ConvEnt_xval_%s.pkl' % (model_name.replace('/','-'), '_'.join([str(lr) for lr in learning_rates])))
pickle.dump(all_val_accs, open(save_fname, 'wb'))

Beginning grid search for ConvEnt over 1 parameter combination(s)!

TRAINING MODEL: bs=32, lr=7.5e-06
Beginning fold 1/8...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[0] Validation results:
{
  accuracy: 
    0.5370370370370371,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[1] Validation results:
{
  accuracy: 
    0.5370370370370371,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[2] Validation results:
{
  accuracy: 
    0.5277777777777778,
}


[3] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[3] Validation results:
{
  accuracy: 
    0.5833333333333334,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[4] Validation results:
{
  accuracy: 
    0.5555555555555556,
}


[5] Beginning epoch.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[0] Validation results:
{
  accuracy: 
    0.5,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[1] Validation results:
{
  accuracy: 
    0.5,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[2] Validation results:
{
  accuracy: 
    0.5,
}


[3] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[3] Validation results:
{
  accuracy: 
    0.5,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[4] Validation results:
{
  accuracy: 
    0.5,
}


[5] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...


...

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[0] Validation results:
{
  accuracy: 
    0.4675324675324675,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[1] Validation results:
{
  accuracy: 
    0.4675324675324675,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[2] Validation results:
{
  accuracy: 
    0.4935064935064935,
}


[3] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[3] Validation results:
{
  accuracy: 
    0.5454545454545454,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[4] Validation results:
{
  accuracy: 
    0.5974025974025974,
}


[5] Beginning epoch.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[0] Validation results:
{
  accuracy: 
    0.4835164835164835,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[1] Validation results:
{
  accuracy: 
    0.4835164835164835,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[2] Validation results:
{
  accuracy: 
    0.5274725274725275,
}


[3] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[3] Validation results:
{
  accuracy: 
    0.5384615384615384,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[4] Validation results:
{
  accuracy: 
    0.5274725274725275,
}


[5] Beginning epoch.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[0] Validation results:
{
  accuracy: 
    0.5204081632653061,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[1] Validation results:
{
  accuracy: 
    0.5510204081632653,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[2] Validation results:
{
  accuracy: 
    0.5510204081632653,
}


[3] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[3] Validation results:
{
  accuracy: 
    0.5714285714285714,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:02s.
[4] Validation results:
{
  accuracy: 
    0.5204081632653061,
}


[5] Beginning epoch.

##Re-Train Best Model from Cross-Validation

In [22]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.train import train_epoch
from www.model.eval import evaluate, save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir
from collections import Counter

# Re-train the model with the best parameters from the grid search/cross-validation (with all folds)
xval_fnames = []
xval_fnames.append(save_fname.split('/')[-1])

xval_results = Counter()
for fname in xval_fnames:
  xval_results += pickle.load(open(os.path.join(GOOGLE_DRIVE_PATH, 'saved_models/', fname), 'rb'))

batch_size, learning_rate, epochs = xval_results.most_common(1)[0][0]
epochs += 1

# Set up model
if 'mnli' not in mode:
  model = model_class.from_pretrained(model_name,
                                      cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
else:
  config = config_class.from_pretrained(model_name.replace('-mnli',''),
                                  num_labels=3,
                                  cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
  model = model_class.from_pretrained(model_name,
                                      config=config,
                                      cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
  config.num_labels = 2
  model.num_labels = 2
  model.classifier = cls_head_class(config=config) # Need to bring in a classification head for only 2 labels

model.cuda()
device = model.device

train_sampler = RandomSampler(ConvEnt_train_tensor)
train_dataloader = DataLoader(ConvEnt_train_tensor, sampler=train_sampler, batch_size=batch_size)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps = total_steps)

for epoch in range(epochs):
  print('[%s] Beginning epoch...' % str(epoch))
  epoch_loss, _ = train_epoch(model, optimizer, train_dataloader, device, seg_mode=True if 'roberta' not in mode else False)

print('[%s] Saving model checkpoint...' % str(epoch))
model_param_str = get_model_dir(model_name.replace('/','-'), 'ConvEnt', batch_size, learning_rate, epoch) + '_xval'
output_dir = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', model_param_str)
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
model = model.module if hasattr(model, 'module') else model
model.save_pretrained(output_dir)
tokenizer.save_vocabulary(output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
[1] Beginning epoch...
[2] Beginning epoch...
[3] Beginning epoch...
[4] Beginning epoch...
[5] Beginning epoch...
[6] Beginning epoch...
[7] Beginning epoch...
[7] Saving model checkpoint...


('drive/My Drive/CSE 595/FinalProject/saved_models/bert-large-uncased_ConvEnt_32_1e-05_7_xval/vocab.txt',)

##Test Models on Conversational Entailment

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.eval import evaluate, save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir

best_model = eval_model_dir


best_model = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', best_model)
# print(best_model)
# Load the model
model = model_class.from_pretrained(best_model)
model.cuda()
device = model.device

# Select appropriate dataset
if 'cloze' in best_model:
  subtask = 'cloze'
elif 'order' in best_model:
  subtask = 'order'

test_sampler = SequentialSampler(ConvEnt_test_tensor)
test_dataloader = DataLoader(ConvEnt_test_tensor, sampler=test_sampler, batch_size=128)
test_dataset_name = '%s_%s' % ('ConvEnt', 'test')
test_ids = [str(ex['example_id']) for ex in ConvEnt_test]

print('Testing model: %s.' % best_model.split('/')[-1])

results, preds, labels = evaluate(model, test_dataloader, device, [(accuracy_score, 'accuracy')], seg_mode=True if 'roberta' not in mode else False)
save_results(results, best_model, test_dataset_name)
save_preds(test_ids, labels, preds, best_model, test_dataset_name)

print('Results:')
print_dict(results)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Testing model: bert-large-uncased_ConvEnt_32_7.5e-06_7_xval.
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:01s.
Results:
{
  accuracy: 
    0.5755813953488372,
}




##Coherence Checks on Conversational Entailment

###Load and Featurize Span Data

In [38]:
from www.dataset.featurize import add_bert_features_ConvEnt, get_tensor_dataset
from www.dataset.prepro import get_ConvEnt_spans
import pickle
seq_length = 128

merged_file = os.path.join(GOOGLE_DRIVE_PATH, 'data/ConvEnt_test_annotation_merged2.json')
ConvEnt_test = json.load(open(merged_file))

ConvEnt_test = add_bert_features_ConvEnt(ConvEnt_test, tokenizer, seq_length, add_segment_ids=True)

if debug:
  ConvEnt_test = ConvEnt_test[:10]

# Some of the annotated examples are no longer in the test set :(
# ConvEnt_test = [ex for ex in ConvEnt_test if ex['id'] in test_ids]

# Make span versions of the datasets
ConvEnt_test_spans = get_ConvEnt_spans(ConvEnt_test)

# Add BERT features
ConvEnt_test_tensor = get_tensor_dataset(ConvEnt_test, label_key='label', add_segment_ids=True)
ConvEnt_test_spans_tensor = get_tensor_dataset(ConvEnt_test_spans, label_key='label', add_segment_ids=True)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

###Load the Trained Model

In [39]:
probe_model = eval_model_dir
probe_model = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', probe_model)

# Load the model
model = model_class.from_pretrained(probe_model)
if torch.cuda.is_available():
  model.cuda()
device = model.device

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [40]:
from www.model.eval import load_preds
from www.utils import print_dict

preds_base = {}
preds_base['test'] = load_preds(os.path.join(probe_model, 'preds_ConvEnt_test.tsv'))
print(preds_base['test'].keys())

dict_keys(['73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '272', '273', '274', '275', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '593', '594', '595', '596', '597', '598', '599', '600', '601', '602', '603', '604', '605', '606', '607', '608', '609', '610', '611', '612', '613', '614', '615', '616', '617', '618', '619', '620', '730', '731', '732', '733', '734', '735', '736', '737', '738', '739', '740', '741', '742', '743', '744', '745', '746', '747', '748', '749', '750', '751', '752', '753', '754', '755', '808', '809', '810', '811', 

###Check a Model

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from www.model.eval import evaluate, save_results, save_preds, list_comparison
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
metrics = [(accuracy_score, 'accuracy'), (precision_score, 'precision'), (recall_score, 'recall'), (f1_score, 'f1')]
import numpy as np
from www.utils import print_dict

def is_polarized(smax, thres):
  return (abs(smax[0] - smax[1]) >= thres)

print('Testing model: %s.' % probe_model)

all_results = {}
p = 'test'

p_dataset = ConvEnt_test_spans
p_tensor_dataset = ConvEnt_test_spans_tensor
p_sampler = SequentialSampler(p_tensor_dataset)
p_dataloader = DataLoader(p_tensor_dataset, sampler=p_sampler, batch_size=512)
p_dataset_name = '%s_spans_%s' % ('ConvEnt', p)
p_dataset_name_co = '%s_consistent_%s' % ('ConvEnt', p)
p_dataset_name_bp = '%s_breakpoints_%s' % ('ConvEnt', p)
p_dataset_name_ev = '%s_evidence_%s' % ('ConvEnt', p)
p_dataset_name_coh = '%s_coherent_%s' % ('ConvEnt', p)
p_ids = [str(ex['example_id']) for ex in ConvEnt_test_spans]
p_labels = [ex['label'] for ex in ConvEnt_test_spans]

# Get span preds and save metrics
results, preds, labels = evaluate(model, p_dataloader, device, metrics, seg_mode=True)
save_results(results, probe_model, p_dataset_name)
save_preds(p_ids, labels, preds, probe_model, p_dataset_name)

# Convert substory preds into breakpoint preds for each example
ids_base = [str(ex['example_id']) for ex in ConvEnt_test]

id_to_pred = {k: v for k,v in zip(p_ids, preds)}
id_to_label = {k: v for k,v in zip(p_ids, p_labels)}

preds_entailment = []
labels_entailment = []
preds_consistent = []
preds_breakpoint = []
labels_breakpoint = []
preds_evidence = []
labels_evidence = []
span_accuracies = []
span_accuracies_strict = []
preds_coherent = []

for i, exid in enumerate(ids_base):
  ex = ConvEnt_test[i]
  ex['length'] = len(ex['turns'])

  label_entailment = preds_base[p][exid]['label']
  pred_entailment = preds_base[p][exid]['pred']
  labels_entailment.append(label_entailment)
  preds_entailment.append(pred_entailment)

  # Get ground truth breakpoint and evidence
  label_breakpoint = ex['conflict_pair'][1] if ex['conflict_pair'] is not None and len(ex['conflict_pair']) > 0 else 0
  labels_breakpoint.append(label_breakpoint)
  if label_breakpoint > 0:
    label_ev = ex['conflict_pair'][0]
  else:
    label_ev = -1
  labels_evidence.append(label_ev)

  # Check consistency - any span that entails the hypothesis' superspans should also entail
  pred_consistent = True
  span_accuracy = 0.0
  span_accuracy_strict = 0.0
  pred_coherent = True

  no_spans = 0
  for sp1 in range(ex['length']):
    if not pred_consistent:
      break

    for sp2 in range(sp1, ex['length']):
      if not pred_consistent:
        break

      span_pred = id_to_pred[exid + '-sp%s:%s' % (str(sp1), str(sp2))]
      span_label = id_to_label[exid + '-sp%s:%s' % (str(sp1), str(sp2))]

      if span_pred == span_label:
        span_accuracy += 1.0
        if label_entailment == pred_entailment:
            span_accuracy_strict += 1.0
      else:
        pred_coherent = False
      no_spans += 1
      # print('%s:%s\t%s\t(%s, %s)' % (str(sp1), str(sp2), str(span_pred), str(span_prob[0]), str(span_prob[1])))

      if span_pred == 1:
        if pred_entailment == 1:
          for sp3 in range(sp1+1):
            if not pred_consistent:
              break

            for sp4 in range(sp2, ex['length']):
              if not pred_consistent:
                break

              sspan_pred = id_to_pred[exid + '-sp%s:%s' % (str(sp3), str(sp4))]

              if sspan_pred == 0:
                pred_consistent = False
                break
        elif pred_entailment == 0:
          pred_consistent = False

  preds_consistent.append(1 if pred_consistent else 0)
  span_accuracies.append(span_accuracy / no_spans)
  span_accuracies_strict.append(span_accuracy_strict / no_spans)
  preds_coherent.append(1 if pred_coherent else 0)

  # Check pred. breakpoint (verifiability) - will be first sentence where the model prediction becomes polarized, i.e., confidence > threshold
  pred_breakpoint = 0 # For now, 0 means -1, i.e., stories are entirely plausible - this shouldn't happen but it will (inconsistent?)
  for ss in range(1, ex['length']):
    if id_to_pred[exid + '-sp%s:%s' % (str(0), str(ss))] == 1:
      pred_breakpoint = ss
      break
  preds_breakpoint.append(pred_breakpoint)

  # Check pred. evidence (verifiability)
  if pred_breakpoint > 0:
    pred_evidence = -1
    for ss in range(0, pred_breakpoint+1):
      if id_to_pred[exid + '-sp%s:%s' % (str(0), str(ss))] == 1:
        pred_evidence = ss
  else:
    pred_evidence = -1 # This should never happen - it would be inconsistent if it did
  preds_evidence.append(pred_evidence)

# Calculate tiered accuracy for model
acc = 0
acc_con = 0
acc_con_vbp = 0
acc_con_vbp_vev = 0
no_ex = len(ids_base)
for p_plaus, l_plaus, con, p_bp, l_bp, p_ev, l_ev in zip(preds_entailment, labels_entailment, preds_consistent, preds_breakpoint, labels_breakpoint, preds_evidence, labels_evidence):
  # Accuracy
  if p_plaus == l_plaus:
    acc += 1

    # Consistency
    if con == 1:
      acc_con += 1

      # Verifiability (breakpoint)
      if p_bp == l_bp:
        acc_con_vbp += 1

        # Verifiability (evidence)
        if p_ev == l_ev:
          acc_con_vbp_vev += 1

acc /= no_ex
acc_con /= no_ex
acc_con_vbp /= no_ex
acc_con_vbp_vev /= no_ex

# all_results['acc'] = acc
# all_results['acc_con'] = acc_con
# all_results['acc_con_vbp'] = acc_con_vbp
# all_results['acc_con_vbp_vev'] = acc_con_vbp_vev
# all_results['span_accuracy'] = np.mean(span_accuracies)

all_results['lenient_coherence'] = np.mean(span_accuracies_strict)
all_results['strict_coherence'] = np.mean(preds_coherent)

best_preds_entailment = preds_entailment
best_preds_consistent = preds_consistent
best_preds_breakpoint = preds_breakpoint
best_preds_evidence = preds_evidence
best_preds_coherent = preds_coherent

print('\nPARTITION: %s' % p)
print_dict(all_results)

# Save preds for breakpoint and evidence
save_preds(ids_base, np.array(labels_breakpoint), best_preds_breakpoint, probe_model, p_dataset_name_bp)
save_preds(ids_base, np.array(labels_evidence), best_preds_evidence, probe_model, p_dataset_name_ev)
save_preds(ids_base, np.array([1 for p in best_preds_coherent]), best_preds_coherent, probe_model, p_dataset_name_coh)

p_dataset_name_agg = '%s_tiers_agg_nostates_lenient_%s' % ('ConvEnt', p)
save_results(all_results, probe_model, p_dataset_name_agg)

Testing model: drive/My Drive/CSE 595/FinalProject/saved_models/bert-large-uncased_ConvEnt_32_7.5e-06_7_xval.
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:05s.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



PARTITION: test
{
  lenient_coherence: 
    0.33255912653997466,
  strict_coherence: 
    0.2441860465116279,
}




#BERT + Graph Encodings

## Load Dataset

In [ ]:
import xml.etree.ElementTree as ET
import pickle
cache_train = os.path.join(GOOGLE_DRIVE_PATH, 'data/ConvEnt_train_resplit.json')
cache_dev = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_resplit.json')
cache_test = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_test_resplit.json')
ConvEnt_train = json.load(open(cache_train))
ConvEnt_dev = json.load(open(cache_dev))
ConvEnt_test = json.load(open(cache_test))

# Combine train and dev and do cross-validation
cache_folds = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_folds.pkl') # Folds used for results presented in paper
ConvEnt_train = ConvEnt_train + ConvEnt_dev
train_sources = list(set([ex['dialog_source'] for ex in ConvEnt_train]))
print("Reserved %s dialog sources for training and validation." % len(train_sources))

no_folds = 8
if not os.path.exists(cache_folds):
  folds = []
  for k in range(no_folds):
    folds.append(np.random.choice(train_sources, size=5, replace=False))
    train_sources = [s for s in train_sources if s not in folds[-1]]
  assert len(train_sources) == 0
  print(folds)
  pickle.dump(folds, open(cache_folds, 'wb'))
else:
  folds = pickle.load(open(cache_folds, 'rb'))

Reserved 40 dialog sources for training and validation.


In [ ]:
print('train examples:', len(ConvEnt_train))
print('dev examples:', len(ConvEnt_dev))
print('test examples:', len(ConvEnt_test))

train examples: 703
dev examples: 110
test examples: 172


In [ ]:
# if debug:
#   ConvEnt_train = ConvEnt_train[:10]
#   ConvEnt_dev = ConvEnt_dev[:10]
#   ConvEnt_test = ConvEnt_test[:10]

In [ ]:
# print('train examples:', len(ConvEnt_train))
# print('dev examples:', len(ConvEnt_dev))
# print('test examples:', len(ConvEnt_test))

## Named Entity Recognition

In [ ]:
def coref_resolve(text):
    nlp1 = spacy.load('en_core_web_trf')
    nlp1.add_pipe('coreferee')
    doc1 = nlp1(text)
    tok_list = list(token.text for token in doc1)
    c = 0
    for chain in doc1._.coref_chains:
        for mention in chain:
            res1 = [doc1._.coref_chains.resolve(doc1[i]) for i in mention]
            res = list(filter((None).__ne__, res1))
            if len(res) != 0:
                if len(res[0]) == 1:
                    tok_list[mention[0] + c] = str(res[0][0])
                elif len(res[0]) > 1:
                    tok_list[mention[0] + c] = str(res[0][0])
                    for j in range(1, len(res[0])):
                        tok_list.insert(mention[0] + c + j, str(res[0][j]))
                        c = c + 1
    textres = " ".join(tok_list)
    return textres


def extract_entities_and_relationships(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    relationships = []
    for sent in doc.sents:
      # Iterate through the named entities (people, organizations etc.) in the sentence
      for ent in sent.ents:
          # Check if the entity has a known label and is a person or organization
          if ent.label_ in ["PERSON", "ORG"]:
              # Extract the relationship
              for token in sent:
                  if token.dep_ in ["attr", "nsubj", "dobj"]:
                      relationships.append((ent.text, token.text))

    #Disregard entitity labels
    entities = [ent[0] for ent in entities]

    return entities, relationships

In [ ]:
sentence = "John Doe is a software engineer at XYZ Corporation. He has a degree in Computer Science from ABC University."
sentence = coref_resolve(sentence)
# sentence = "SpeakerB : And , uh , SpeakerB was about , the , the piece of music , the piece of music was about , I think about forty or fifty years old .   And , SpeakerB was incredible , I mean , the parallel , you know , between piece and rap . SpeakerA : Right . SpeakerB : And , um , you , you listen a lot , if you , if you hear a lot of old gospel , uh , uh , especially well , the black gospel .   You know , you will , you know , you can really pick gospel up .   I mean ,"
print(sentence)
nlp = spacy.load('en_core_web_sm')

res_entities, res_relationships = extract_entities_and_relationships(sentence)
print("Entities:", res_entities)
print("Relationships:", res_relationships)

/usr/local/lib/python3.10/dist-packages/spacy_transformers/layers/hf_shim.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fileli

John Doe is a software engineer at XYZ Corporation . Doe has a degree in Computer Science from ABC University .
Entities: ['John Doe', 'XYZ Corporation', 'Computer Science', 'ABC University']
Relationships: [('John Doe', 'Doe'), ('John Doe', 'engineer'), ('XYZ Corporation', 'Doe'), ('XYZ Corporation', 'engineer'), ('Computer Science', 'Doe'), ('Computer Science', 'degree'), ('ABC University', 'Doe'), ('ABC University', 'degree')]


##Get Instanse-Specific Knowledge Graph by Querying Global Knowledge Graph

In [ ]:
!pip install SPARQLWrapper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 12.9 MB/s eta 0:00:00


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

# DBpedia SPARQL endpoint
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

# Function to query DBpedia for information about an entity
def query_dbpedia(entity):
    query = f"""
    SELECT ?subject ?predicate ?object
    WHERE {{
      ?subject rdfs:label "{entity}"@en .
      ?subject ?predicate ?object
    }}
    LIMIT 50
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

In [ ]:
# Function to build an instance-specific knowledge graph
def build_instance_kg(entities):
    kg = {}
    for entity in entities:
        results = query_dbpedia(entity)
        triples = [(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"])
                   for result in results["results"]["bindings"]]
        kg[entity] = triples
    return kg

In [ ]:
# Example entities to query
entities = res_entities

# Build the knowledge graph
instance_kg = build_instance_kg(entities)

# Print the knowledge graph
for entity, triples in instance_kg.items():
    print(f"Entity: {entity}")
    for subject, predicate, obj in triples:
        print(f"  {subject} -> {predicate} -> {obj}")
    print()

Entity: John Doe
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/ontology/Person
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/class/yago/WikicatAnonymityPseudonyms
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/class/yago/WikicatCollectivePseudonyms
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/class/yago/WikicatLanguageVarietiesAndStyles
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/class/yago/Ability105616246
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/class/yago/Abstraction100002137
  http://dbpedia.org/resource/John_Doe -> http://www.w3.org/1999/02/22-rdf-syntax-ns#type -> http://dbpedia.org/c

In [ ]:
for value in instance_kg.values():
  print(len(value))

50
0
18
0


##Generate Node Embeddings From Instance-Specific Knowledge Graph

In [ ]:
!pip install node2vec networkx

In [ ]:
import networkx as nx
from node2vec import Node2Vec

# Construct a NetworkX graph from the instance-specific knowledge graph
def construct_graph(instance_kg, relationships):
    G = nx.DiGraph()
    for entity, triples in instance_kg.items():
        for subject, predicate, obj in triples:
            G.add_node(entity, label=entity)
            G.add_node(obj)
            G.add_edge(subject, obj, label=predicate)
    for rel in relationships:
        G.add_node(rel[0], label=rel[0])
        G.add_node(rel[1], label=rel[1])
        G.add_edge(rel[0], rel[1])
    return G

# Generate node embeddings using node2vec
def generate_node_embeddings(G, entity_nodes, dimensions=768, walk_length=30, num_walks=200, workers=4):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)

    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # Retrieve embeddings only for the entity nodes
    node_embeddings = {str(node): model.wv[str(node)] for node in entity_nodes if str(node) in model.wv}
    return node_embeddings


In [ ]:
relationships = res_relationships

# Construct the graph
G = construct_graph(instance_kg, relationships)

# Generate node embeddings only for the entity nodes
node_embeddings = generate_node_embeddings(G, entities)

# Print embeddings of the entity nodes
for node, embedding in node_embeddings.items():
    print(f"Node: {node}, Embedding: {embedding[:5]}...")  # Print only the first 5 dimensions for brevity

Computing transition probabilities:   0%|          | 0/63 [00:00<?, ?it/s]

Node: John Doe, Embedding: [-7.1464196e-06  9.6979871e-04 -2.6296737e-04 -2.8108250e-04
  1.0065751e-03]...
Node: XYZ Corporation, Embedding: [ 0.0004201  -0.00075284  0.00056785  0.00120331 -0.00107555]...
Node: Computer Science, Embedding: [-0.00133304 -0.00013657 -0.0011246   0.00019095  0.00033662]...
Node: ABC University, Embedding: [-2.2274052e-04  1.1672511e-03 -4.4540322e-04  8.5892412e-04
  6.8043759e-05]...


## Featurize BERT Embeddings with Node Embeddings

In [ ]:
import torch
from torch.utils.data import TensorDataset
import spacy
import progressbar
import numpy as np

# Creates tensor dataset from featurized dataset
def get_tensor_dataset_plus_node(dataset, label_key='plausible', add_segment_ids=False):
  # print(len(dataset[0]['input_ids']))
  # print(len(dataset[1]['input_ids']))
  all_input_ids = torch.tensor([ex['input_ids'] for ex in dataset])
  all_input_node_embeddings_idx = torch.tensor([ex['input_node_embeddings_idx'] for ex in dataset])
  all_input_mask = torch.tensor([ex['input_mask'] for ex in dataset])
  all_label_ids = torch.tensor([ex[label_key] for ex in dataset], dtype=torch.long)

  if 'segment_ids' in dataset[0]:
    all_segment_ids = torch.tensor([ex['segment_ids'] for ex in dataset])
    tensor_dataset = TensorDataset(all_input_ids, all_input_node_embeddings_idx, all_input_mask, all_label_ids, all_segment_ids)
  else:
    tensor_dataset = TensorDataset(all_input_ids, all_input_node_embeddings_idx, all_input_mask, all_label_ids)
  return tensor_dataset

# Tokenize, numericalize, and pad an input dataset
def add_bert_features_ConvEnt_plus_node(dataset, tokenizer, seq_length, node_embeddings_from_storage=False, node_embeddings_path=None, add_segment_ids=False):

  # Get stored node embeddings if available
  if node_embeddings_from_storage:
    print('Loading node embeddings from storage...')
    node_embeddings_dict = pickle.load(open(node_embeddings_path, 'rb'))
  else:
    node_embeddings_dict = {}

  for i, ex in enumerate(dataset):
    exid = ex['id']
    ex['example_id'] = exid

    # Numericalize the data
    dialog = ' '.join(['Speaker%s: %s' % (turn['speaker'], turn['text']) for turn in ex['turns']])
    inputs = tokenizer.encode_plus(dialog,
                                    text_pair=ex['hypothesis'],
                                    add_special_tokens=True,
                                    max_length=seq_length,
                                    truncation=True)
    input_ids = inputs['input_ids']
    if 'token_type_ids' in inputs:
      token_type_ids = inputs['token_type_ids']

    if not node_embeddings_from_storage:
      # Coreference dialog for better entity/relationship extraction
      coref_dialog = coref_resolve(dialog)

      # Entity/relationship extracton
      entities, relationships = extract_entities_and_relationships(coref_dialog)

      if(len(entities) > 0):

        # Build the local knowledge graph
        instance_kg = build_instance_kg(entities)

        relevant_entities = False
        for value in instance_kg.values():
          if len(value) > 0:
            relevant_entities = True
            break

        if len(relationships) > 0:
          relevant_entities = True

        if relevant_entities:

          print(exid, len(entities))
          print(coref_dialog)

          # Construct the graph
          G = construct_graph(instance_kg, relationships)

          # Generate node embeddings only for the entity nodes
          node_embeddings = generate_node_embeddings(G, entities)

        else:
          node_embeddings = {}

      else:

        node_embeddings = {}

      # Store node embeddings in dictionary, so that node embeddings do not have to be recreated
      node_embeddings_dict[i] = node_embeddings

    # Don't want to truncate any data
    assert not('num_truncated_tokens' in inputs and inputs['num_truncated_tokens'] > 0)
    assert len(input_ids) <= seq_length

    # Pad to sequence length of 128
    padding_length = seq_length - len(input_ids)
    input_length = len(input_ids)
    input_ids = input_ids + ([0] * padding_length)
    attention_mask = [1] * input_length + ([0] * padding_length) # Mask will zero out padding tokens
    if 'token_type_ids' in inputs:
      token_type_ids = token_type_ids + [0] * padding_length

    assert len(input_ids) == len(attention_mask) == seq_length

    dataset[i]['input_ids'] = input_ids
    dataset[i]['input_mask'] = attention_mask
    if 'token_type_ids' in inputs:
      dataset[i]['segment_ids'] = token_type_ids
    dataset[i]['input_node_embeddings_idx'] = [i] * seq_length

  return dataset, node_embeddings_dict


In [ ]:
import pickle
seq_length = 128

nlp = spacy.load('en_core_web_sm')

cache_embeddings_train = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_node_embeddings_train.pkl')
cache_embeddings_dev = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_node_embeddings_dev.pkl')
cache_embeddings_test = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_node_embeddings_test.pkl')

# ConvEnt_train, node_embeddings_train = add_bert_features_ConvEnt_plus_node(ConvEnt_train, tokenizer, seq_length, add_segment_ids=True)
ConvEnt_dev, node_embeddings_dev = add_bert_features_ConvEnt_plus_node(ConvEnt_dev, tokenizer, seq_length, add_segment_ids=True, node_embeddings_from_storage=True, node_embeddings_path=cache_embeddings_dev)

# Cache node_embeddings_dev
if not os.path.exists(cache_embeddings_dev):
  print('Caching node embeddings...')
  # print(node_embeddings_dev)
  pickle.dump(node_embeddings_dev, open(cache_embeddings_dev, 'wb'))

ConvEnt_test, node_embeddings_test = add_bert_features_ConvEnt_plus_node(ConvEnt_test, tokenizer, seq_length, add_segment_ids=True, node_embeddings_from_storage=True, node_embeddings_path=cache_embeddings_test)

# Cache node_embeddings_test
if not os.path.exists(cache_embeddings_test):
  print('Caching node embeddings...')
  # print(node_embeddings_test)
  pickle.dump(node_embeddings_test, open(cache_embeddings_test, 'wb'))

ConvEnt_train, node_embeddings_train = add_bert_features_ConvEnt_plus_node(ConvEnt_train, tokenizer, seq_length, add_segment_ids=True, node_embeddings_from_storage=True, node_embeddings_path=cache_embeddings_train)

# Cache node_embeddings_train
if not os.path.exists(cache_embeddings_train):
  print('Caching node embeddings...')
  # print(node_embeddings_train)
  pickle.dump(node_embeddings_train, open(cache_embeddings_train, 'wb'))


ConvEnt_train_folds = [[] for _ in range(no_folds)]
ConvEnt_dev_folds = [[] for _ in range(no_folds)]
for k in range(no_folds):
  ConvEnt_train_folds[k] = [ex for ex in ConvEnt_train if ex['dialog_source'] not in folds[k]]
  ConvEnt_dev_folds[k] = [ex for ex in ConvEnt_train if ex['dialog_source'] in folds[k]]

  if debug:
    ConvEnt_train_folds[k] = ConvEnt_train_folds[k][:10]
    ConvEnt_dev_folds[k] = ConvEnt_dev_folds[k][:10]

if debug:
  ConvEnt_train = ConvEnt_train[:10]
  ConvEnt_dev = ConvEnt_dev[:10]
  ConvEnt_test = ConvEnt_test[:10]

ConvEnt_train_tensor = get_tensor_dataset_plus_node(ConvEnt_train, label_key='label', add_segment_ids=True)
ConvEnt_test_tensor = get_tensor_dataset_plus_node(ConvEnt_test, label_key='label', add_segment_ids=True)

# Training sets for each validation fold
ConvEnt_train_folds_tensor = [get_tensor_dataset_plus_node(ConvEnt_train_folds[k], label_key='label', add_segment_ids=True) for k in range(no_folds)]
ConvEnt_dev_folds_tensor = [get_tensor_dataset_plus_node(ConvEnt_dev_folds[k], label_key='label', add_segment_ids=True) for k in range(no_folds)]


Loading node embeddings from storage...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Loading node embeddings from storage...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Loading node embeddings from storage...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
print('train examples:', len(ConvEnt_train))
print('dev examples:', len(ConvEnt_dev))
print('test examples:', len(ConvEnt_test))

train examples: 703
dev examples: 110
test examples: 172


In [ ]:
trainNodeEmbeddingCount = 0
for item in node_embeddings_train.values():
  if bool(item):
    trainNodeEmbeddingCount += 1
print("Percentage of train with node embeddings:", trainNodeEmbeddingCount / len(node_embeddings_train))

devNodeEmbeddingCount = 0
for item in node_embeddings_dev.values():
  if bool(item):
    devNodeEmbeddingCount += 1
print("Percentage of dev with node embeddings:", devNodeEmbeddingCount / len(node_embeddings_dev))

testNodeEmbeddingCount = 0
for item in node_embeddings_test.values():
  if bool(item):
    testNodeEmbeddingCount += 1
print("Percentage of test with node embeddings:", testNodeEmbeddingCount / len(node_embeddings_test))

Percentage of train with node embeddings: 0.3741109530583215
Percentage of dev with node embeddings: 0.2818181818181818
Percentage of test with node embeddings: 0.46511627906976744


## Augment Embeddings

In [ ]:
def augment_embedding(input_ids, token_embeddings, node_embeddings, tokenizer):

  batch_size = input_ids.size(0)
  seq_length = input_ids.size(1)
  device = token_embeddings.device

  # Initialize a list to hold the augmented embeddings
  augmented_embeddings = []

  # Loop through each item in batch
  for i in range(batch_size):
      instance_embeddings = []

      # Loop through each token in the input
      for token, token_emb in zip(input_ids[i], token_embeddings[i]):
          token_str = tokenizer.decode([token]).strip()

          # Check if the token corresponds to an entity with a node embedding
          if token_str in node_embeddings:
              node_embedding = node_embeddings[token_str]
              combined_embedding = torch.cat((token_emb, node_embedding), dim=0)
          else:
              # If no corresponding node embedding, pad with zeros or use the original embedding
              node_embedding = torch.zeros(768, device=device)
              combined_embedding = torch.cat((token_emb, node_embedding), dim=0)
          instance_embeddings.append(combined_embedding)

      # Add instance imbedding to batch
      instance_embeddings = torch.stack(instance_embeddings)
      augmented_embeddings.append(instance_embeddings)

  # Convert list to tensor
  augmented_embeddings = torch.stack(augmented_embeddings)

  return augmented_embeddings


##Augmented BERT

In [ ]:
# Define a custom BERT model that handles augmented embeddings
class AugmentedBERTModel(torch.nn.Module):
    def __init__(self, bert_model_class, tokenizer, model_name, cache_dir=None): # Pass model name and class
        super(AugmentedBERTModel, self).__init__()
        # Load pretrained bert model
        self.bert_model = bert_model_class.from_pretrained(model_name, cache_dir=cache_dir)
        self.tokenizer = tokenizer
        self.fc = torch.nn.Linear(1792, 1024)  # Reduce dimensions back to 1024
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)  # Move the model to the device

    def forward(self, input_ids, node_embeddings, attention_mask=None, token_type_ids=None, labels=None):
        # Get the token embeddings from BERT's embedding layer
        token_embeddings = self.bert_model.bert.embeddings(input_ids)

        # print(input_ids.shape)
        # print(token_embeddings.shape)

        augmented_embeddings = augment_embedding(input_ids, token_embeddings, node_embeddings, self.tokenizer)

        # print(augmented_embeddings.shape)

        # Pass the augmented embeddings through the initial layers
        hidden_states = self.fc(augmented_embeddings)

        # Get the outputs from the BERT model
        outputs = self.bert_model(
            inputs_embeds=hidden_states,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )
        return outputs

In [ ]:
config_class = BertConfig
emb_class = BertModel
model_class = AugmentedBERTModel

## Train & Evaluate Functions

In [ ]:
import time
import torch
from www.utils import format_time
import numpy as np
from transformers import RobertaForMultipleChoice
import progressbar
from www.model.eval import evaluate_tiered
from sklearn.metrics import accuracy_score, f1_score

# Train a PyTorch model for one epoch
def train_epoch_plus_node(model, optimizer, train_dataloader, device, node_embedding_dict, list_output=False, num_outputs=1, span_mode=False, seg_mode=False, classifier=None, multitask_idx=None):
  t0 = time.time()

  if not list_output:
    total_loss = 0
  else:
    total_loss = [0 for _ in range(num_outputs)]

  # Training mode
  model.train()

  if len(train_dataloader) * train_dataloader.batch_size >= 2500:
    progress_update = True
  else:
    progress_update = False

  for step, batch in enumerate(train_dataloader):
    # Progress update
    if progress_update and step % 50 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('\t(%s) Starting batch %s of %s.' % (elapsed, str(step), str(len(train_dataloader))))

    input_ids = batch[0].to(device)

    # Get node embedding index and move embeddings to device
    #TODO - NEED TO LOOP THROUGH ALL ITEMS IN BATCH, ADD ALL NODE EMEDDINGS
    input_node_embeddings_idx = batch[1][0][0].item()
    input_node_embeddings = node_embedding_dict[input_node_embeddings_idx]

    # print(input_node_embeddings)
    for item in input_node_embeddings:
      input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)

    input_mask = batch[2].to(device)
    labels = batch[3].to(device)
    segment_ids = batch[4].to(device)

    # Forward pass
    model.zero_grad()
    out = model(input_ids,
                input_node_embeddings,
                token_type_ids=segment_ids,
                attention_mask=input_mask,
                labels=labels)

    loss = out[0]

    # Backward pass
    if not list_output:
      total_loss += loss.item()
      loss.backward()
    else:
      for o in range(num_outputs):
        total_loss[o] += loss[o].item()
        loss[o].backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping

    optimizer.step()

  if list_output:
    return list(np.array(total_loss) / len(train_dataloader)), model
  else:
    return total_loss / len(train_dataloader), model


In [ ]:
import numpy as np
import time
from www.utils import format_time, read_tsv
import torch
import json
import os
import progressbar
from www.dataset.ann import att_default_values

def compute_metrics(preds, labels, metrics):
  metr = {}
  for m, m_name in metrics:
    if m_name in ['accuracy', 'confusion_matrix']:
      metr[m_name] = m(labels, preds) # Assume each metric m will be a function of (y_true, y_pred)
    else:
      metr[m_name] = m(labels, preds, average='macro')
  return metr

# Run evaluation for a PyTorch model
def evaluate_plus_node(model, eval_dataloader, device, node_embedding_dict, metrics, list_output=False, num_outputs=1, span_mode=False, seg_mode=False, return_softmax=False, multilabel=False, lm=False):
  print('\tBeginning evaluation...')

  t0 = time.time()

  model.zero_grad()
  model.eval()

  all_labels = None
  if not list_output:
    all_preds = None
    if return_softmax:
      all_logits = None
  else:
    all_preds = [np.array([]) for _ in range(num_outputs)]

  print('\t\tRunning prediction...')
  # Get preds from model
  for batch in eval_dataloader:

    # Move to GPU
    batch = tuple(t.to(device) for t in batch)

    input_ids, input_node_embeddings_idx, input_mask, labels, segment_ids = batch

    # Get node embedding index and move embeddings to device
    #TODO - NEED TO LOOP THROUGH ALL ITEMS IN BATCH, ADD ALL NODE EMEDDINGS
    input_node_embeddings_idx = input_node_embeddings_idx[0][0].item()
    input_node_embeddings = node_embedding_dict[input_node_embeddings_idx]

    for item in input_node_embeddings:
      input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)

    input_node_embeddings = node_embedding_dict[input_node_embeddings_idx]

    for item in input_node_embeddings:
      input_node_embeddings[item] = input_node_embeddings[item].to(device)

    with torch.no_grad():
      out = model(input_ids,
                  input_node_embeddings,
                  token_type_ids=segment_ids,
                  attention_mask=input_mask)

    label_ids = labels.to('cpu').numpy()
    if all_labels is None:
      all_labels = label_ids
    else:
      all_labels = np.concatenate((all_labels, label_ids), axis=0)
    # print(all_labels.shape)

    logits = out[0]
    if list_output: # This is broken, do not use
      metr = {}
      for o in range(num_outputs):
        preds = torch.argmax(logits[o], dim=1).detach().cpu().numpy()
        all_preds[o] = np.concatenate((all_preds[o], preds))
        metr_o = compute_metrics(all_preds[o], all_labels, metrics)
        for k, v in metr_o.items():
          metr[str(o) + '_' + k] = v
    else:
      preds = torch.argmax(logits, dim=1 if not lm else 2).detach().cpu().numpy()
      # print(preds.shape)
      if all_preds is None:
        all_preds = preds
      else:
        all_preds = np.concatenate((all_preds, preds))
      # print(all_preds.shape)
      if return_softmax:
        logits = torch.softmax(logits, dim=1 if not lm else 2).detach().cpu().numpy()
        if all_logits is None:
          all_logits = logits
        else:
          all_logits = np.concatenate((all_logits, logits))

  # Calculate metrics
  print('\t\tComputing metrics...')
  metr = compute_metrics(all_preds, all_labels, metrics)

  print('\tFinished evaluation in %ss.' % str(format_time(time.time() - t0)))

  if not return_softmax:
    return metr, all_preds, all_labels
  else:
    # Warning: this is not supported in list_output mode
    return metr, all_preds, all_labels, all_logits

## Train Model

In [ ]:
batch_sizes = [config_batch_size]
learning_rates = [config_lr]
epochs = config_epochs
eval_batch_size = 128

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.eval import save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir
from collections import Counter

seed_val = 22 # Save random seed for reproducibility
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

assert len(batch_sizes) == 1
train_fold_sampler = [RandomSampler(f) for f in ConvEnt_train_folds_tensor]
train_fold_dataloader = [DataLoader(f, sampler=train_fold_sampler[i], batch_size=batch_sizes[0]) for i, f in enumerate(ConvEnt_train_folds_tensor)]

dev_fold_sampler = [SequentialSampler(f) for f in ConvEnt_dev_folds_tensor]
dev_fold_dataloader = [DataLoader(f, sampler=dev_fold_sampler[i], batch_size=eval_batch_size) for i, f in enumerate(ConvEnt_dev_folds_tensor)]

all_val_accs = Counter()
print('Beginning grid search for ConvEnt over %s parameter combination(s)!' % (str(len(batch_sizes) * len(learning_rates))))
for bs in batch_sizes:
  for lr in learning_rates:
    print('\nTRAINING MODEL: bs=%s, lr=%s' % (str(bs), str(lr)))

    for k in range(no_folds):
      print('Beginning fold %s/%s...' % (str(k+1), str(no_folds)))

      # Set up model
      if 'mnli' not in mode:
        model = model_class(BertForSequenceClassification, tokenizer, model_name, cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
        # model = model_class.from_pretrained(model_name, cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
      else:
        config = config_class.from_pretrained(model_name.replace('-mnli',''),
                                        num_labels=3,
                                        cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
        model = model_class.from_pretrained(model_name,
                                            config=config,
                                            cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
        config.num_labels = 2
        model.num_labels = 2
        model.classifier = cls_head_class(config=config) # Need to bring in a classification head for only 2 labels

      model.cuda()
      device = model.device

      # Set up optimizer
      optimizer = AdamW(model.parameters(), lr=lr)
      total_steps = len(train_fold_dataloader[k]) * epochs
      scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps = total_steps)

      for epoch in range(epochs):
        # Train the model for one epoch
        print('[%s] Beginning epoch...' % str(epoch))

        epoch_loss, _ = train_epoch_plus_node(model, optimizer, train_fold_dataloader[k], device, node_embeddings_train, seg_mode=True)

        # Validate on dev set
        results, _, _ = evaluate_plus_node(model, dev_fold_dataloader[k], device, node_embeddings_train, [(accuracy_score, 'accuracy')], seg_mode=True)
        print('[%s] Validation results:' % str(epoch))
        print_dict(results)

        # Save accuracy
        acc = results['accuracy']
        if (bs, lr, epoch) in all_val_accs:
          all_val_accs[(bs, lr, epoch)] += acc
        else:
          all_val_accs[(bs, lr, epoch)] = acc

      model.cpu()
      del model
      del optimizer
      del results
      del scheduler
      del total_steps

      print('[%s] Finished epoch.' % str(epoch))

for k in all_val_accs:
  all_val_accs[k] /= no_folds

print('Top performing param combos:')
print(all_val_accs.most_common(5))

save_fname = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models/%s_ConvEnt_xval_%s.pkl' % (model_name.replace('/','-'), '_'.join([str(lr) for lr in learning_rates])))
pickle.dump(all_val_accs, open(save_fname, 'wb'))

Beginning grid search for ConvEnt over 1 parameter combination(s)!

TRAINING MODEL: bs=32, lr=7.5e-06
Beginning fold 1/8...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:04s.
[0] Validation results:
{
  accuracy: 
    0.5462962962962963,
}


[1] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[1] Validation results:
{
  accuracy: 
    0.5370370370370371,
}


[2] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[2] Validation results:
{
  accuracy: 
    0.46296296296296297,
}


[3] Beginning epoch...


<ipython-input-36-43102644430d>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)


	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:04s.
[3] Validation results:
{
  accuracy: 
    0.5370370370370371,
}


[4] Beginning epoch...
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[4] Validation results:
{
  accuracy: 
    0.5370370370370371,
}


[5] Beginning epoch...


...

	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[5] Validation results:
{
  accuracy: 
    0.5612244897959183,
}


[6] Beginning epoch...


<ipython-input-36-43102644430d>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)


	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[6] Validation results:
{
  accuracy: 
    0.5306122448979592,
}


[7] Beginning epoch...


<ipython-input-36-43102644430d>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)


	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:03s.
[7] Validation results:
{
  accuracy: 
    0.45918367346938777,
}


[7] Finished epoch.
Top performing param combos:
[((32, 7.5e-06, 2), 0.530557347100321), ((32, 7.5e-06, 1), 0.5297506015299962), ((32, 7.5e-06, 4), 0.5166809074720017), ((32, 7.5e-06, 0), 0.510735352707816), ((32, 7.5e-06, 5), 0.5048005234156226)]


## Re-Train Best Model from Cross-Validation

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.eval import save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir
from collections import Counter

# Re-train the model with the best parameters from the grid search/cross-validation (with all folds)
xval_fnames = []
xval_fnames.append(save_fname.split('/')[-1])

xval_results = Counter()
for fname in xval_fnames:
  xval_results += pickle.load(open(os.path.join(GOOGLE_DRIVE_PATH, 'saved_models/', fname), 'rb'))

# batch_size, learning_rate, epochs = xval_results.most_common(1)[0][0]
batch_size, learning_rate, epochs = 32, 1e-05, 7
epochs += 1

# Set up model
if 'mnli' not in mode:
  model = model_class(BertForSequenceClassification, tokenizer, model_name, cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
  # model = model_class.from_pretrained(model_name, cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
else:
  config = config_class.from_pretrained(model_name.replace('-mnli',''),
                                  num_labels=3,
                                  cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
  model = model_class.from_pretrained(model_name,
                                      config=config,
                                      cache_dir=os.path.join(GOOGLE_DRIVE_PATH, 'cache'))
  config.num_labels = 2
  model.num_labels = 2
  model.classifier = cls_head_class(config=config) # Need to bring in a classification head for only 2 labels

model.cuda()
device = model.device

train_sampler = RandomSampler(ConvEnt_train_tensor)
train_dataloader = DataLoader(ConvEnt_train_tensor, sampler=train_sampler, batch_size=batch_size)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps = total_steps)

for epoch in range(epochs):
  print('[%s] Beginning epoch...' % str(epoch))
  epoch_loss, _ = train_epoch_plus_node(model, optimizer, train_dataloader, device, node_embeddings_train, seg_mode=True if 'roberta' not in mode else False)

print('[%s] Saving model checkpoint...' % str(epoch))
model_param_str = get_model_dir(model_name.replace('/','-'), 'ConvEnt', batch_size, learning_rate, epoch) + '_xval'
print(model_param_str)
output_dir = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', model_param_str)
if not os.path.exists(output_dir):
  os.makedirs(output_dir)
model = model.module if hasattr(model, 'module') else model

# Saving AugmentedModel does not work without custom save/from_pretrained function, rather just have to run all prior to load best model
# model.save_pretrained(output_dir)
tokenizer.save_vocabulary(output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

[0] Beginning epoch...
[1] Beginning epoch...
[2] Beginning epoch...
[3] Beginning epoch...


<ipython-input-48-43102644430d>:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_node_embeddings[item] = torch.tensor(input_node_embeddings[item]).to(device)


[4] Beginning epoch...
[5] Beginning epoch...
[6] Beginning epoch...
[7] Beginning epoch...
[7] Saving model checkpoint...
bert-large-uncased_ConvEnt_32_1e-05_7_xval


('drive/My Drive/CSE 595/FinalProject/saved_models/bert-large-uncased_ConvEnt_32_1e-05_7_xval/vocab.txt',)

## Test Models on Conversational Entailment

In [23]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
from www.model.eval import evaluate, save_results, save_preds
from sklearn.metrics import accuracy_score
from www.utils import print_dict, get_model_dir

best_model = eval_model_dir


best_model = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', best_model)

# Saving AugmentedModel does not work without custome from_pretrained function, rather just have to run all prior to load best model
# print(best_model)
# Load the model
# model = model_class.from_pretrained(best_model)
# model.cuda()
# device = model.device

# Select appropriate dataset
if 'cloze' in best_model:
  subtask = 'cloze'
elif 'order' in best_model:
  subtask = 'order'

test_sampler = SequentialSampler(ConvEnt_test_tensor)
# test_dataloader = DataLoader(ConvEnt_test_tensor, sampler=test_sampler, batch_size=128)
test_dataloader = DataLoader(ConvEnt_test_tensor, sampler=test_sampler, batch_size=1)
test_dataset_name = '%s_%s' % ('ConvEnt', 'test')
test_ids = [str(ex['example_id']) for ex in ConvEnt_test]

print('Testing model: %s.' % best_model.split('/')[-1])

results, preds, labels = evaluate_plus_node(model, test_dataloader, device, node_embeddings_test, [(accuracy_score, 'accuracy')], seg_mode=True if 'roberta' not in mode else False)
save_results(results, best_model, test_dataset_name)
save_preds(test_ids, labels, preds, best_model, test_dataset_name)

print('Results:')
print_dict(results)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Testing model: bert-large-uncased_ConvEnt_32_1e-05_7_xval.
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:01s.
Results:
{
  accuracy: 
    0.6162790697674418,
}




## Coherence Checks on Conversational Entailment

### Load and Featurize Span Data

In [ ]:
from www.dataset.prepro import get_ConvEnt_spans
import pickle
seq_length = 128

merged_file = os.path.join(GOOGLE_DRIVE_PATH, 'data/ConvEnt_test_annotation_merged2.json')
ConvEnt_test = json.load(open(merged_file))

cache_embeddings_test_merged = os.path.join(GOOGLE_DRIVE_PATH,'data/ConvEnt_dev_node_embeddings_test_merged.pkl')

ConvEnt_test, node_embeddings_test_merged = add_bert_features_ConvEnt_plus_node(ConvEnt_test, tokenizer, seq_length, add_segment_ids=True, node_embeddings_from_storage=True, node_embeddings_path=cache_embeddings_test_merged)

# Cache node_embeddings_test
if not os.path.exists(cache_embeddings_test_merged):
  print('Caching node embeddings...')
  # print(node_embeddings_test_merged)
  pickle.dump(node_embeddings_test_merged, open(cache_embeddings_test_merged, 'wb'))


if debug:
  ConvEnt_test = ConvEnt_test[:10]

# Some of the annotated examples are no longer in the test set :(
# ConvEnt_test = [ex for ex in ConvEnt_test if ex['id'] in test_ids]

# Make span versions of the datasets
ConvEnt_test_spans = get_ConvEnt_spans(ConvEnt_test)

# Add BERT features
ConvEnt_test_tensor = get_tensor_dataset_plus_node(ConvEnt_test, label_key='label', add_segment_ids=True)
ConvEnt_test_spans_tensor = get_tensor_dataset_plus_node(ConvEnt_test_spans, label_key='label', add_segment_ids=True)

Loading node embeddings from storage...


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

### Load the Trained Model

In [ ]:
probe_model = eval_model_dir
probe_model = os.path.join(GOOGLE_DRIVE_PATH, 'saved_models', probe_model)

# Saving AugmentedModel does not work without custom save/from_pretrained function, rather just have to run all prior to load best model
# Load the model
# model = model_class.from_pretrained(probe_model)
# if torch.cuda.is_available():
#   model.cuda()
# device = model.device

In [ ]:
from www.model.eval import load_preds
from www.utils import print_dict

preds_base = {}
preds_base['test'] = load_preds(os.path.join(probe_model, 'preds_ConvEnt_test.tsv'))
print(preds_base['test'].keys())

dict_keys(['73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '272', '273', '274', '275', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '593', '594', '595', '596', '597', '598', '599', '600', '601', '602', '603', '604', '605', '606', '607', '608', '609', '610', '611', '612', '613', '614', '615', '616', '617', '618', '619', '620', '730', '731', '732', '733', '734', '735', '736', '737', '738', '739', '740', '741', '742', '743', '744', '745', '746', '747', '748', '749', '750', '751', '752', '753', '754', '755', '808', '809', '810', '811', 

### Check a Model

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from www.model.eval import save_results, save_preds, list_comparison
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
metrics = [(accuracy_score, 'accuracy'), (precision_score, 'precision'), (recall_score, 'recall'), (f1_score, 'f1')]
import numpy as np
from www.utils import print_dict

def is_polarized(smax, thres):
  return (abs(smax[0] - smax[1]) >= thres)

print('Testing model: %s.' % probe_model)

all_results = {}
p = 'test'

p_dataset = ConvEnt_test_spans
p_tensor_dataset = ConvEnt_test_spans_tensor
p_sampler = SequentialSampler(p_tensor_dataset)
p_dataloader = DataLoader(p_tensor_dataset, sampler=p_sampler, batch_size=512)
p_dataset_name = '%s_spans_%s' % ('ConvEnt', p)
p_dataset_name_co = '%s_consistent_%s' % ('ConvEnt', p)
p_dataset_name_bp = '%s_breakpoints_%s' % ('ConvEnt', p)
p_dataset_name_ev = '%s_evidence_%s' % ('ConvEnt', p)
p_dataset_name_coh = '%s_coherent_%s' % ('ConvEnt', p)
p_ids = [str(ex['example_id']) for ex in ConvEnt_test_spans]
p_labels = [ex['label'] for ex in ConvEnt_test_spans]

# Get span preds and save metrics
results, preds, labels = evaluate_plus_node(model, p_dataloader, device, node_embeddings_test, metrics, seg_mode=True)
save_results(results, probe_model, p_dataset_name)
save_preds(p_ids, labels, preds, probe_model, p_dataset_name)

# Convert substory preds into breakpoint preds for each example
ids_base = [str(ex['example_id']) for ex in ConvEnt_test]

id_to_pred = {k: v for k,v in zip(p_ids, preds)}
id_to_label = {k: v for k,v in zip(p_ids, p_labels)}

preds_entailment = []
labels_entailment = []
preds_consistent = []
preds_breakpoint = []
labels_breakpoint = []
preds_evidence = []
labels_evidence = []
span_accuracies = []
span_accuracies_strict = []
preds_coherent = []

for i, exid in enumerate(ids_base):
  ex = ConvEnt_test[i]
  ex['length'] = len(ex['turns'])

  label_entailment = preds_base[p][exid]['label']
  pred_entailment = preds_base[p][exid]['pred']
  labels_entailment.append(label_entailment)
  preds_entailment.append(pred_entailment)

  # Get ground truth breakpoint and evidence
  label_breakpoint = ex['conflict_pair'][1] if ex['conflict_pair'] is not None and len(ex['conflict_pair']) > 0 else 0
  labels_breakpoint.append(label_breakpoint)
  if label_breakpoint > 0:
    label_ev = ex['conflict_pair'][0]
  else:
    label_ev = -1
  labels_evidence.append(label_ev)

  # Check consistency - any span that entails the hypothesis' superspans should also entail
  pred_consistent = True
  span_accuracy = 0.0
  span_accuracy_strict = 0.0
  pred_coherent = True

  no_spans = 0
  for sp1 in range(ex['length']):
    if not pred_consistent:
      break

    for sp2 in range(sp1, ex['length']):
      if not pred_consistent:
        break

      span_pred = id_to_pred[exid + '-sp%s:%s' % (str(sp1), str(sp2))]
      span_label = id_to_label[exid + '-sp%s:%s' % (str(sp1), str(sp2))]

      if span_pred == span_label:
        span_accuracy += 1.0
        if label_entailment == pred_entailment:
            span_accuracy_strict += 1.0
      else:
        pred_coherent = False
      no_spans += 1
      # print('%s:%s\t%s\t(%s, %s)' % (str(sp1), str(sp2), str(span_pred), str(span_prob[0]), str(span_prob[1])))

      if span_pred == 1:
        if pred_entailment == 1:
          for sp3 in range(sp1+1):
            if not pred_consistent:
              break

            for sp4 in range(sp2, ex['length']):
              if not pred_consistent:
                break

              sspan_pred = id_to_pred[exid + '-sp%s:%s' % (str(sp3), str(sp4))]

              if sspan_pred == 0:
                pred_consistent = False
                break
        elif pred_entailment == 0:
          pred_consistent = False

  preds_consistent.append(1 if pred_consistent else 0)
  span_accuracies.append(span_accuracy / no_spans)
  span_accuracies_strict.append(span_accuracy_strict / no_spans)
  preds_coherent.append(1 if pred_coherent else 0)

  # Check pred. breakpoint (verifiability) - will be first sentence where the model prediction becomes polarized, i.e., confidence > threshold
  pred_breakpoint = 0 # For now, 0 means -1, i.e., stories are entirely plausible - this shouldn't happen but it will (inconsistent?)
  for ss in range(1, ex['length']):
    if id_to_pred[exid + '-sp%s:%s' % (str(0), str(ss))] == 1:
      pred_breakpoint = ss
      break
  preds_breakpoint.append(pred_breakpoint)

  # Check pred. evidence (verifiability)
  if pred_breakpoint > 0:
    pred_evidence = -1
    for ss in range(0, pred_breakpoint+1):
      if id_to_pred[exid + '-sp%s:%s' % (str(0), str(ss))] == 1:
        pred_evidence = ss
  else:
    pred_evidence = -1 # This should never happen - it would be inconsistent if it did
  preds_evidence.append(pred_evidence)

# Calculate tiered accuracy for model
acc = 0
acc_con = 0
acc_con_vbp = 0
acc_con_vbp_vev = 0
no_ex = len(ids_base)
for p_plaus, l_plaus, con, p_bp, l_bp, p_ev, l_ev in zip(preds_entailment, labels_entailment, preds_consistent, preds_breakpoint, labels_breakpoint, preds_evidence, labels_evidence):
  # Accuracy
  if p_plaus == l_plaus:
    acc += 1

    # Consistency
    if con == 1:
      acc_con += 1

      # Verifiability (breakpoint)
      if p_bp == l_bp:
        acc_con_vbp += 1

        # Verifiability (evidence)
        if p_ev == l_ev:
          acc_con_vbp_vev += 1

acc /= no_ex
acc_con /= no_ex
acc_con_vbp /= no_ex
acc_con_vbp_vev /= no_ex

# all_results['acc'] = acc
# all_results['acc_con'] = acc_con
# all_results['acc_con_vbp'] = acc_con_vbp
# all_results['acc_con_vbp_vev'] = acc_con_vbp_vev
# all_results['span_accuracy'] = np.mean(span_accuracies)

all_results['lenient_coherence'] = np.mean(span_accuracies_strict)
all_results['strict_coherence'] = np.mean(preds_coherent)

best_preds_entailment = preds_entailment
best_preds_consistent = preds_consistent
best_preds_breakpoint = preds_breakpoint
best_preds_evidence = preds_evidence
best_preds_coherent = preds_coherent

print('\nPARTITION: %s' % p)
print_dict(all_results)

# Save preds for breakpoint and evidence
save_preds(ids_base, np.array(labels_breakpoint), best_preds_breakpoint, probe_model, p_dataset_name_bp)
save_preds(ids_base, np.array(labels_evidence), best_preds_evidence, probe_model, p_dataset_name_ev)
save_preds(ids_base, np.array([1 for p in best_preds_coherent]), best_preds_coherent, probe_model, p_dataset_name_coh)

p_dataset_name_agg = '%s_tiers_agg_nostates_lenient_%s' % ('ConvEnt', p)
save_results(all_results, probe_model, p_dataset_name_agg)

Testing model: drive/My Drive/CSE 595/FinalProject/saved_models/bert-large-uncased_ConvEnt_32_1e-05_7_xval.
	Beginning evaluation...
		Running prediction...
		Computing metrics...
	Finished evaluation in 0:00:05s.

PARTITION: test
{
  lenient_coherence: 
    0.4137883624203733,
  strict_coherence: 
    0.3430232558139535,
}


